In [2]:
import pandas as pd

In [114]:
ludnosc = []
for i in range(1, 17):
    ludnosc.append(pd.read_excel('../data/raw/gus/LUD_bilans_ludnosci_31-12-2011_fixed.xls', sheet_name=i))
ludnosc=pd.concat(ludnosc).dropna(axis=0)
powiaty = ludnosc[ludnosc[ludnosc.columns[0]].str.startswith('pow.')]
powiaty = powiaty[powiaty.columns[[0,2]]].copy()
powiaty.columns = ['Nazwa', 'Ludnosc']
powiaty['Nazwa']=powiaty['Nazwa'].str.replace('pow\.','').str.replace('m\.','').str.replace('st\. ','').str.replace('Wałbrzych','wałbrzyski').str.strip()
powiaty['Ludnosc']=powiaty['Ludnosc'].apply(int)
del ludnosc
ludnosc=powiaty.groupby('Nazwa')['Ludnosc'].sum()

In [86]:
mobility = pd.read_excel("../data/raw/gus/l_macierz_2014_03_18-1.xls", dtype=str)
mobility['SourcePowiat']=mobility['Powiat zamieszkania'].str.replace('Powiat','').str.replace('m\.','').str.replace('st\. ','').str.strip().replace('Wałbrzych', 'wałbrzyski')
mobility['TargetPowiat']=mobility['Poawiat pracy'].str.replace('Powiat','').str.replace('m\.','').str.replace('st\. ','').str.strip().replace('Wałbrzych', 'wałbrzyski')
mobility['Liczba osób']=mobility['Liczba osób'].astype(int)
od_matrix = mobility.groupby(['SourcePowiat', 'TargetPowiat'])['Liczba osób'].sum()

In [42]:
#wget https://www.gis-support.pl/downloads/Powiaty.zip -O ../data/raw/powiaty.zip

In [45]:
import geopandas as gpd

In [117]:
powiat_shape = gpd.read_file('../data/raw/powiaty.zip')
powiat_shape['Nazwa']=powiat_shape['JPT_NAZWA_'].str.replace('powiat','').str.replace('Wałbrzych','wałbrzyski').str.strip()
powiat_shape = powiat_shape.dissolve(by='Nazwa')

In [124]:
od_powiat_names = set(mobility['TargetPowiat'].values).union(mobility['SourcePowiat'].values)
shape_names = set(powiat_shape.index.values)
ludnosc_names = set(ludnosc.index.values)

In [133]:
# Make indices agree

In [134]:
len(od_powiat_names), len(shape_names), len(ludnosc_names)

(369, 369, 369)

In [135]:
len(shape_names.intersection(od_powiat_names)), len(shape_names.intersection(ludnosc_names)), len(od_powiat_names.intersection(ludnosc_names))

(369, 369, 369)

In [140]:
import numpy as np
def get_distance(centroid1, centroid2):
    c1=tuple(centroid1.coords)[0]
    c2=tuple(centroid2.coords)[0]
    return np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)

In [166]:
dists = pd.Series({k: get_distance(powiat_shape.loc[k[0]].geometry.representative_point(), powiat_shape.loc[k[1]].geometry.representative_point()) for k in od_matrix.index.values}, name='r')/1000

In [170]:
city_pair_matrix = pd.DataFrame([od_matrix, dists]).T.round(0)+1

In [154]:
get_distance(powiat_shape.loc['Warszawa'].geometry.representative_point(), powiat_shape.loc['Wrocław'].geometry.representative_point())

307941.12682343205